In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Auswerten eines Schach-Endspiels

Zum Vergleich, dass die berechneten Züge der KI eine gute Lösung für das Gewinnen einer Endspielsituation sind, wird das Programm Stockfish verwendet. Es stellt eine Schach-Engine dar, die nach einem zuvor definierten Muster optimale Schachzüge berechnet.

## Importe

Für die Durchführung der Tests und der Auswertung eines Schach-Endspiels gegen eine andere KI werden die hier stehenden Bibliotheken benötigt:

* ``chess``: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* ``stockfish``: Bibliothek, welche eine Python Schnittstelle für die Stockfish-Enginge anbietet.
* ``clear_output`` aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen
verwendet wird.
* ``display`` aus der IPython-display-Library: Wird zum grafischen Anzeigen der Schachbretter verwendet.
* ``json``: Wird zum Speichern der Ergebnisse verwendet.
* ``time``: Wird zum Verzögern von Ausgaben verwendet.

In [ ]:
import chess
import stockfish
from IPython.display import clear_output
from IPython.display import display
import json
import time

## Konfiguration

Zu Beginn müssen für den Vergleich unterschiedliche Variablen definiert werden. Diese beinhalten eine Fen, die für ein gegebenes Szenario überprüft werden soll, den Pfad zu der berechneten $S_n$ Sequenz und einen Pfad zu der Stockfish-Installation.

In [ ]:
FEN = "4k3/8/8/8/8/8/8/R3K3 w - - 0 1"
S_N_FILE = "S_n_seq_12_01.json"
STOCKFISH_PATH = "./stockfish_14.1/stockfish_14.1_win_x64_avx2.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die mit Hilfe des Notebooks ``calculation.ipynb`` bestimmt werden. Diese wurden bereits in einer JSON-Datei serialisiert abgespeichert. Für dieses Notebook werden alle FENs gelesen und daraus Board-Objekte erstellt. Dies geschieht mit der Funktion ``load_s_n_sequence``. Diese erhält den Dateipfad (``filepath``) als Parameter, um aus der Datei die $S_n$ Mengen auslesen zu können.

In [ ]:
def load_s_n_sequence(filepath):
    s_n_sequence_object = []
    s_n_sequence_tuples = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_object = []
        tmp_set = set()
        for board in item:
            chess_board = chess.Board(board)
            tmp_object.append(chess_board)
            tmp_set.add((chess_board.turn,chess_board.__str__()))
        s_n_sequence_tuples.append(tmp_set)
        s_n_sequence_object.append(tmp_object)
    f.close()
    return s_n_sequence_tuples, s_n_sequence_object

In [ ]:
load_s_n_sequence(S_N_FILE)

## Die Züge für die KI ermitteln

Der erste Schritt, um einen Zug für die KI zu ermitteln, liegt im Finden der momentanen Spielsituation. Dies wird mit der Funktion ``find_board_in_sequence`` erreicht. Sie dursucht eine gegebene $S_n$ Sequenz nach einem übergebenen Schachbrett. Die Parameter hierfür lauten wie folgt:

* ``situation``: Das Board (als Objekt), welches gefunden werden soll.
* sequence: Die $S_n$ Sequenz, in welcher das Board gesucht wird.

Das Ergebnis der Funktion kann insgesamt zwei unterschiedliche Formen annehmen:
* Ein Tupel mit $S_n$ Index (z.B. $S_3$) und Board-Index (z.B. 100).Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
* Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Das Ergebniss wird mithilfe von dem anschließenden Algorithmus bestimmt:
1. Über die $S_n$ Sequenz iterieren.
2. Über jedes Board in einem spezifischem $S_n$ iterieren.
3. Das Board mit dem ``situation`` Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [ ]:
def find_board_in_sequence(situation, sequence):
    board_tuple = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        if board_tuple in sequence[i]:
            return i
    return -1

Nachdem das Brett in einer $S_n$ Menge gefunden wurde, ist der nächste Schritt den passenden Zug für die KI zu bestimmen. Zu diesem Zweck wurde die Funktion ``find_next_move`` definiert. Diese berechnet für ein übergebenes Board den idealen Spielzug. Weiterhin wird aus Effizienzgründen schon die Position des nächsten Bretts in der $S_n$ Sequenz zurückgegeben. Für diese Berechnung benötigt ``find_next_move`` nachkommende Argumente:

* ``curr_board``: Das Board, für welches der nächste Spielzug berechnet werden soll.
* ``s_index``: Das $n$ eines $S_n$, in welchem sich ``curr_board`` befindet. Wird aus Effizienzgründen übergeben.
* ``s_n_sequence``: Die Liste mit allen $S_n$

Als Ergebnis liefert die Funktion den nächsten Move oder den Wert -1 als Hinweis, dass kein Spielzug berechnet werden konnte.

Der Algorithmus dieser Funktion erstreckt sich folgender Art und Weise:
* Führe alle möglichen Moves durch, bis das Ergebnis-Board in $S_{n-1}$ gefunden wurde.
* Gebe den neuen ``s_index``, ``board_index`` und den gefundenen ``move`` zurück.

In [ ]:
def find_next_move(curr_board, s_index, s_n_sequence):
    STOCKFISH.set_fen_position(curr_board.fen())
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            _tmp = find_board_in_sequence(curr_board, [s_n_sequence[s_index - 1]])
            s_index_tmp = s_index - 1
            if _tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, move
            curr_board.pop()

        return -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        s_index = find_board_in_sequence(curr_board, s_n_sequence[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, move

## Vergleich zwischen Stockfish und KI

Für den Vergleich werden die Anzahlen der Züge zwischen Stockfish und der selbstgeschriebenen KI miteinander verglichen.

Bei der Berechnung der Züge für die KI wird die Funktion ``calculate_all_moves`` verwendet. Diese berechnet in einer Schleife alle Moves ausgehend von einer Fen, bis die KI gewonnen hat. Die Berechnung verwendet hierzu folgende Parameter:

* ``fen``: Die Fen der Spielsituation, die berechnet werden soll.
* ``s_n_sequence``: Die Liste der $S_n$, welche für die Figurenkonstellation berechnet wurde.

Das Ergebnis der Ausführung ist eine Liste aller Moves, welche für das Board berechnet wurde oder der Wert None als Hinweis, dass kein Sieg erzielt werden konnte.

Die Liste wurde durch den anschließenden Algorithmus bestimmt:
* Einordnung der übergebenen Fen in der $S_n$ Sequenz finden.
* Solange $n > 0$ in einer Schleife den nächsten Spielzug berechnen.
* Die Liste der errechneten Spielzüge zurückgeben.

In [ ]:
def calculate_all_moves(fen, s_n_sequence):
    moves = []

    board = chess.Board(fen)

    s_index = find_board_in_sequence(board, s_n_sequence)

    while s_index > 0:
        s_index, next_move = find_next_move(board, s_index, s_n_sequence)
        board.push(next_move)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

Nachdem eine Liste aller Züge für die KI bestimmt wurden, gilt dies gleichermaßen für die Berechnung von Stockfish umzusetzen. Hierfür erhält die Funktion ``stockfish_movelist`` den Parameter:
* ``fen``: Die FEN des Boardes, für welches alle Züge berechnet werden sollen.

Nach der Ausführung erhält man ebenso eine List von Spielzügen.

Die Liste wurde mit einem leicht abgeänderten Algorithmus bestimmt:
* Solange das Spiel nicht beendet ist einen weiteren Zug berechnen.
* Am Ende alle Züge zurückgeben.

In [ ]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves


Die Funktion `move_in_sequence` überprüft für ein gegebenes Brett, ob dieses auch in der gegebenen S_n Menge vorhanden ist.
Hierfür werden folgende Parameter verwendet:
- `board_to_check`: Das Brett, das in der Menge `sequence` vorhanden sein soll.
- `sequence`: Die Menge, in der `board_to_check` auffindbar sein soll.

Als Ergebnis wird ein boolescher Wert zurückgegeben.

In [ ]:
def move_in_sequence(board_to_check, sequence):
    if find_board_in_sequence(board_to_check, [sequence]) != -1:
        return True
    else:
        return False

Bei `sequenz_index` wird für jedes Board einer S_n Menge überprüft, ob Stockfish oder die zuvor definierten Mengen das
Board effizienter lösen. Dabei gilt es für den Vergleich zu beachten, dass folgende Vorgehensweise verwendet wird.
- `Stockfish`: Für sowohl schwarz, als auch für weiß, werden die Züge mit der Funktion `Stockfish().get_best_move()` bestimmt.
- `KI`: Das n, das überprüft wird, wird für den Vergleich genutzt, da dies den schlechtesten Fall darstellt.

Die Methode, die für den Vergleich mit Stockfish angewandt wurde, birgt die Gefahr, dass durch Stockfish für schwarz
nicht der beste Zug ausgewählt wird und dadurch das Ergebnis beeinflusst. Diese Gefahr erhöht sich vor allem dann, wenn
sich die Züge bis zu einem Schachmatt erhöhen, da bei jedem Zug von schwarz die Gefahr besteht, dass nicht der beste Zug
ausgewählt wird.

Die Funktion erhält als Parameter:
- `sequenz_index`: Die Nummer der Menge, für die alle Bretter überprüft werden sollen.
(Bsp.: S_10 -> `sequence_index = 10`)

In der Konsole wird nach den Berechnungen dem Nutzer angezeigt, zu wie viel % Stockfish und die selbstgeschriebene KI
bessere Ergebnisse erzielt hat. Außerdem wird noch angezeigt zu welchem Teil sowohl Stockfish, als auch die KI gleiche
Ergebnisse erzielt haben.

In [ ]:
def get_board_from_tuple(tuple, sequence):
    for board in sequence:
        cmp_tuple = (board.turn, board.__str__())
        # print(cmp_tuple)
        if tuple == cmp_tuple:
            return board.copy()

In [ ]:
def compare_sequence_stockfish(sequence_index, s_n_sequence_tuple, s_n_object):
    s_n = s_n_sequence_tuple[sequence_index]
    # print(sequence_index)
    move_count_list = []
    diff_list = []
    length = len(s_n)
    r = 0
    for board_ in s_n:
        seq_index = sequence_index
        cmp_board = get_board_from_tuple(board_, s_n_object[seq_index]).copy()
        # print(seq_index, b_index)
        move_count = 0
        # print(move_in_sequence(board_it, s_n))
        if board_ in s_n:
            cur = board_
            while seq_index > 0:
                # White
                #print("Round")
                #print(board_it)
                #print(seq_index)
                board_to_use = get_board_from_tuple(cur, s_n_object[seq_index])
                #print(board_to_use)
                if board_to_use.turn:
                    # print("White")
                    for p_move in board_to_use.legal_moves:
                        # print(p_move)
                        board_to_use.push(p_move)
                        # print("B function: ", b_index, seq_index)
                        s_tmp = find_board_in_sequence(board_to_use, [s_n_sequence_tuple[seq_index-1]])
                        # print("A function: ", b_index, seq_index)
                        if s_tmp != -1:
                            seq_index -= 1
                            # print(b_index, seq_index)
                            move_count += 1
                            cur = (board_to_use.turn, board_to_use.__str__())
                            break
                        board_to_use.pop()
                # Black
                else:
                    # print("Black")
                    STOCKFISH.set_fen_position(board_to_use.fen())
                    nxt_move = chess.Move.from_uci(STOCKFISH.get_best_move())
                    # print(nxt_move)
                    board_to_use.push(nxt_move)
                    move_count += 1
                    seq_index = find_board_in_sequence(board_to_use, s_n_sequence_tuple[:seq_index])
                    cur = (board_to_use.turn, board_to_use.__str__())
                    # print(b_index, seq_index)
        else:
            print("Couldn't find board!")

        cmp_move_count = 0
        while not cmp_board.is_game_over():
            STOCKFISH.set_fen_position(cmp_board.fen())
            nxt_move = chess.Move.from_uci(STOCKFISH.get_best_move())
            cmp_board.push(nxt_move)
            cmp_move_count += 1
        # print("Moves KI: " + str(move_count), "Moves Stockfish: " + str(cmp_move_count))
        move_count_list.append((move_count, cmp_move_count, move_count - cmp_move_count))
        diff_list.append(move_count - cmp_move_count)
        clear_output()
        print("Comparing S_" + str(sequence_index) + ":")
        print("Compared " + str(r+1) + "/" + str(length))
        r+=1
    ki_better = 0
    stockfish_better = 0
    equal = 0
    for value in diff_list:
        if value == 0:
            equal += 1
        elif value < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    cmp_values = len(diff_list)
    f = open("Stockfish_compare.txt", "a")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better/cmp_values) * 100, 2)) + "% besser\n")
    f.write("Die KI war zu " + str(round((ki_better/cmp_values) * 100, 2)) + "% besser\n")
    f.write("Stockfish und die KI haben zu " + str(round((equal/cmp_values) * 100, 2)) + "% die gleichen Ergebnisse erzielt\n")
    f.close()

Als Beispiel wird hier der Vergleich zwischen der KI und Stockfish für die Menge S_20 vorgenommen.

In [ ]:
S_N_Sequence_Set, S_N_Sequence_Object = load_s_n_sequence(S_N_FILE)

In [ ]:
compare_sequence_stockfish(0, S_N_Sequence_Set, S_N_Sequence_Object)

Ein Testszenario sieht auch vor alle zuvor bestimmten Boards mit dem Lösungsweg von Stockfish zu vergleichen. Hierfür
geht die Funktion `compare_all_sequences` alle S_n-Mengen durch und vergleicht diese jeweils mit der Lösung von
Stockfish.

In [ ]:
def compare_all_sequences(s_n_sequence_tuple, s_n_object):
    for i in range(len(s_n_sequence_tuple)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i, s_n_sequence_tuple, s_n_object)

Für die visuelle Darstellung der Züge wurde die Funktion ``show_movelist`` definiert. Diese zeigt auf einem ``chess.Board`` eine mitgegebene Liste von Zügen. Die Funktionsargumente der Funktion lauten:
* ``fen``: Die Fen des Boardes, für welches alle Züge dargestellt werden sollen.
* ``moves``: Die Liste der Spielzüge, welche dargestellt werden sollen.

In [ ]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)

Der Vergleich zwischen der KI und Stockfish kann nun anhand der Länge der Listen durchgeführt werden. Außerdem können die Züge durch die Funktion ``show_movelist`` nachvollzogen werden.

In [ ]:
compare_all_sequences(S_N_Sequence_Set, S_N_Sequence_Object)

In [ ]:
Moves = calculate_all_moves(FEN, S_N_Sequence_Set)
Stockfish_Moves = stockfish_movelist(FEN)

if Moves is not None:
    print("AI needed " + str(len(Moves)) + " moves to beat Stockfish as Black.")
else:
    print("AI found no way to beat Black.")

print("Stockfish needed " + str(len(Stockfish_Moves)) + " moves to win against itself.")

In [ ]:
show_movelist(FEN, Moves)